## 1. 自注意力机制的学习

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V
$$

- Q：查询矩阵, 表示当前处理的输入元素
- K：键矩阵, 表示输入元素的特征元素
- V：值矩阵, 是么个输入元素所有包含的信息, 当计算出与查询的匹配程度（通过键）后，最终需要提取活聚合的信息
- dk：键向量的维度
- QKT：查询与键的点积
- softmax：用于将点积结果归一化为概率分布

In [45]:
import numpy as np

# 输入序列
tokens = [
    "The", "cat", "sat", "on", "the", "mat"
]


Q = K = V =[
    [0.1, 0.2, 0.3, 0.4], # The
    [0.5, 0.6, 0.7, 0.8], # cat
    [0.9, 1.0, 1.1, 1.2], # sat
    [1.3, 1.4, 1.5, 1.6], # on
    [1.7, 1.8, 1.9, 2.0], # the
    [2.1, 2.2, 2.3, 2.4]  # mat  
]

Q = np.array(Q)
K = np.array(K)
V = np.array(V)

# Q[1]与 K 的每行做点积
scores = np.dot(Q[1], K.T)
print("scores", scores)


sqrt_dk = np.sqrt(K.shape[1])
print('sqrt_dk',sqrt_dk)

scale_scores = scores / sqrt_dk
print('scale_scores',scale_scores)


def softmax(x):
    # 将一组实数转换为0-1 之间的概率分布
    return np.exp(x) / np.sum(np.exp(x), axis=0)

weights = softmax(scale_scores)
print('weights',weights)

output = np.dot(weights, V)
print('output',output)
# 这就是 cat 注意力机制的最终输出

scores [0.7  1.74 2.78 3.82 4.86 5.9 ]
sqrt_dk 2.0
scale_scores [0.35 0.87 1.39 1.91 2.43 2.95]
weights [0.0315077  0.05299682 0.08914212 0.14993952 0.25220241 0.42421143]
output [1.62438656 1.72438656 1.82438656 1.92438656]


## 2. 多头注意力

多头注意力通过将查询、键、值向量分割成多个较小的矩阵，称为头，然后分别进行自注意力的计算，最后将所有头的输出合并起来。
优势：
1. 捕获词元之间的不同语义和语法关系。
2. 增加模型的容量和表达性
3. 提高模型并行化和效率
4. 
$$
\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h)W^O
$$

其中，每个注意力头 $\text{head}_i$ 的计算公式为：

$$
\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)
$$

注意力函数 $\text{Attention}(Q, K, V)$ 使用缩放点积注意力计算：

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

## 符号说明
- $W_i^Q$,  $W_i^K$ , $W_i^V$: 用于第i个注意力头的查询、键和值的权重矩阵。
- $W^O$: 用于将所有注意力头输出连接后的权重矩阵。
- $h$: 注意力头的数量。
- $d_k$: 键向量的维度。
- $text{Concat}$: 将所有头的输出进行连接操作。